In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from urllib.parse import quote_plus
from datetime import datetime, timedelta, time
import pandas as pd

In [2]:
def db_select(query):
    host = "rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net"
    database = "db1"
    user = "test_user"
    password = 'j2M{CnnFq@'
    password_escaped = quote_plus(password)
    port = 6432
    db_type = 'postgresql'

    try:
        connection_string = f"postgresql+psycopg2://{user}:{password_escaped}@{host}:{port}/{database}"
        engine = create_engine(connection_string)

        df = pd.read_sql_query(query, con=engine)

        engine.dispose()

        return df

    except Exception as e:
        print(f"Ошибка {e}")
        return None

In [3]:
messages = db_select("SELECT * FROM test.chat_messages;")
manager_details = db_select("SELECT * FROM test.managers;")

In [4]:
messages.head(10)

,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",incoming_chat_message,37531455,0,1730016782
3,"""01jb75bkprk0d7hht1g5vy1pm4""",incoming_chat_message,37553371,0,1730038583
4,"""01jb3t5xprbph1c6pym8ervxpm""",incoming_chat_message,37548675,0,1729926199
5,"""01jb478n40pk0c0v1zsk3wtwfr""",incoming_chat_message,37550015,0,1729939920
6,"""01jb3t5vr8rfzv8j2ny519nqzk""",incoming_chat_message,37427859,0,1729926197
7,"""01jb3z6s5gv240x8qg8kj8927v""",incoming_chat_message,37054671,0,1729931470
8,"""01jb3t5rtg6s3eks367rm33rb0""",incoming_chat_message,37546971,0,1729926194
9,"""01jb3t5rtghb56gcbvktr7txx5""",incoming_chat_message,37549489,0,1729926194


In [5]:
manager_details.head(10)

,mop_id,name_mop,rop_id
0,6645315,Гюнель и Илина,1
1,6744792,Юля и Наташа,1
2,10262505,Вика и Марго,1
3,10262513,Ира и Варя,1
4,10465254,Настя и Даша,1
5,11448610,Вика и Катя,1
6,11490270,Аня и Ксюша,1
7,11490278,Ксюша и Джамиля,1
8,6780177,Даша и Даша,2
9,7436430,Софа и Алина,2


In [21]:
messages = messages.copy()
messages['message_type'] = messages['type'].replace('incoming_chat_message', 'client').replace('outgoing_chat_message', 'manager')
messages['message_time'] = pd.to_datetime(messages['created_at'], unit='s')
messages_sorted = messages.sort_values(by='message_time').reset_index(drop=True)

In [22]:
def align_to_business_hours(timestamp):
    business_start = time(9, 30)
    if timestamp.time() < business_start:
        return timestamp.replace(hour=9, minute=30, second=0, microsecond=0)
    return timestamp

messages_sorted['message_time'] = messages_sorted['message_time'].apply(align_to_business_hours)

messages_sorted['block_id'] = ((messages_sorted['message_type'] != messages_sorted['message_type'].shift()) |
                               (messages_sorted['entity_id'] != messages_sorted['entity_id'].shift())).cumsum()

In [23]:
first_messages = messages_sorted.groupby('block_id').first().reset_index(drop=True)

first_messages['date'] = first_messages['message_time'].dt.date
first_messages['interval_minutes'] = first_messages.groupby('entity_id')['message_time'].diff().dt.total_seconds() / 60
first_messages['date_difference'] = first_messages.groupby('entity_id')['date'].diff()

In [24]:
# Копируем данные сообщений для вычисления интервалов
filtered_messages = first_messages[(first_messages['message_type'] == 'manager') & (first_messages['interval_minutes'].notna())].copy()
filtered_messages['date_difference'] = filtered_messages['date_difference'].apply(lambda x: x.days if pd.notna(x) else 0)

In [25]:
# Вычисляем интервал времени с учетом выходных
filtered_messages['work_time_interval'] = (
    filtered_messages['interval_minutes'] - 
    filtered_messages['date_difference'] * (34200 / 60)
)

In [26]:
avg_response_time = filtered_messages.groupby("created_by")['work_time_interval'].mean().round(2)

In [27]:
final_manager_data = pd.merge(manager_details, avg_response_time, left_on='mop_id', right_index=True, how='inner')
final_manager_data.rename(columns={'work_time_interval': 'average_response_time'}, inplace=True)
final_manager_data = final_manager_data.sort_values(by='mop_id').reset_index(drop=True)

In [28]:
pd.set_option('display.float_format', '{:.2f}'.format)

print(final_manager_data)

      mop_id             name_mop rop_id  average_response_time
0    6645315       Гюнель и Илина      1                   7.63
1    6744792         Юля и Наташа      1                   9.14
2    6780177          Даша и Даша      2                   3.34
3    7408305        Влада и Настя      3                   4.71
4    7417617     Порхачева Полина      3                   6.18
5    7996770           Лиза и Ева      2                   5.08
6    8558094          Алина и Юля      3                   5.90
7   10262485          Соня и Катя      2                  11.19
8   10262493          Ками и Мила      3                   8.74
9   10262497         Мария и Соня      2                   7.52
10  10262513           Ира и Варя      1                  19.09
11  10465254         Настя и Даша      1                   2.96
12  10465690       Настя и Малика      3                   9.24
13  11188802         Даша и Влада      2                  12.82
14  11241942        Даша и Карина      3